In [33]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json

spark = SparkSession.builder.appName("SparkStreamingDemo").getOrCreate()

# subscribe au topic 'persons'
df_stream = spark.readStream.format("kafka") \
    .option('kafka.bootstrap.servers', 'kafka:9092') \
    .option('subscribe', 'topic-persons') \
    .option('startingOffsets', 'earliest') \
    .load()

query = df_stream.writeStream.format("console").start()
query.awaitTermination()

25/10/07 11:57:12 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-00ba690b-a458-4457-baa8-9d94f2c0c3f1. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/10/07 11:57:12 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/10/07 11:57:13 WARN NetworkClient: [AdminClient clientId=adminclient-31] Connection to node -1 (localhost/127.0.0.1:9092) could not be established. Broker may not be available.


-------------------------------------------
Batch: 0
-------------------------------------------
+----+--------------------+-------------+---------+------+--------------------+-------------+
| key|               value|        topic|partition|offset|           timestamp|timestampType|
+----+--------------------+-------------+---------+------+--------------------+-------------+
|null|[22 7B 5C 22 69 6...|topic-persons|        0|     0|2025-10-07 09:40:...|            0|
|null|[22 7B 5C 22 69 6...|topic-persons|        0|     1|2025-10-07 09:40:...|            0|
|null|[22 7B 5C 22 69 6...|topic-persons|        0|     2|2025-10-07 09:40:...|            0|
|null|[22 7B 5C 22 69 6...|topic-persons|        0|     3|2025-10-07 09:40:...|            0|
|null|[22 7B 5C 22 69 6...|topic-persons|        0|     4|2025-10-07 09:40:...|            0|
|null|[22 7B 5C 22 69 6...|topic-persons|        0|     5|2025-10-07 09:40:...|            0|
|null|[22 7B 5C 22 69 6...|topic-persons|        0|     6

25/10/07 11:57:14 WARN NetworkClient: [AdminClient clientId=adminclient-31] Connection to node -1 (localhost/127.0.0.1:9092) could not be established. Broker may not be available.
25/10/07 11:57:15 WARN NetworkClient: [AdminClient clientId=adminclient-31] Connection to node -1 (localhost/127.0.0.1:9092) could not be established. Broker may not be available.
25/10/07 11:57:16 WARN NetworkClient: [AdminClient clientId=adminclient-31] Connection to node -1 (localhost/127.0.0.1:9092) could not be established. Broker may not be available.
25/10/07 11:57:17 WARN NetworkClient: [AdminClient clientId=adminclient-31] Connection to node -1 (localhost/127.0.0.1:9092) could not be established. Broker may not be available.
25/10/07 11:57:18 WARN KafkaOffsetReaderAdmin: Error in attempt 3 getting Kafka offsets: 
java.util.concurrent.ExecutionException: org.apache.kafka.common.errors.TimeoutException: Timed out waiting for a node assignment. Call: describeTopics
	at java.base/java.util.concurrent.Com

KeyboardInterrupt: 

# Exercice

1) Créer un topic kafka avec KafkaAdminClient.
2) Se connecter en stream sur le topic avec Spark
3) Envoyer les données présentes dans person.json dans le topic avec KafkaProducer


In [34]:
kafka_server = 'kafka:9092'

In [43]:
import json
from kafka.admin import KafkaAdminClient, NewTopic
from kafka import KafkaProducer

topic = 'persons'

admin = KafkaAdminClient(bootstrap_servers=kafka_server)
topic_list = admin.list_topics()
if topic not in topic_list:
    new_topic = NewTopic(name=topic, num_partitions=1, replication_factor=1)
    admin.create_topics(new_topics=[new_topic])

producer = KafkaProducer(bootstrap_servers=[kafka_server], value_serializer=lambda v: json.dumps(v).encode('utf-8'))

with open('data/persons.json', 'r') as file:
    persons = json.load(file)
    for person in persons:
        producer.send(topic, value=person)

    producer.flush()

producer.close()

-------------------------------------------
Batch: 1
-------------------------------------------
-------------------------------------------
Batch: 1
-------------------------------------------
+--------------------+
|               value|
+--------------------+
|{"id": 1, "name":...|
|{"id": 2, "name":...|
|{"id": 3, "name":...|
|{"id": 4, "name":...|
|{"id": 5, "name":...|
|{"id": 6, "name":...|
|{"id": 7, "name":...|
|{"id": 8, "name":...|
|{"id": 9, "name":...|
|{"id": 10, "name"...|
|{"id": 11, "name"...|
|{"id": 12, "name"...|
|{"id": 13, "name"...|
|{"id": 14, "name"...|
|{"id": 15, "name"...|
|{"id": 16, "name"...|
|{"id": 17, "name"...|
|{"id": 18, "name"...|
|{"id": 19, "name"...|
|{"id": 20, "name"...|
+--------------------+
only showing top 20 rows

+----+--------------------+-------+---------+------+--------------------+-------------+
| key|               value|  topic|partition|offset|           timestamp|timestampType|
+----+--------------------+-------+---------+------+

In [44]:
from pyspark.sql import SparkSession
topic = 'persons'
spark = SparkSession.builder.appName("SparkStreamingDemo").getOrCreate()
df_stream = spark.readStream.format("kafka") \
    .option('kafka.bootstrap.servers', kafka_server) \
    .option('subscribe', topic) \
    .option('startingOffsets', 'earliest') \
    .load()

query = df_stream.writeStream.format("console").start()
query.awaitTermination(10)

25/10/07 12:04:56 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
25/10/07 12:04:56 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-1d16e871-822a-4342-878a-b56d775410e1. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/10/07 12:04:56 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+----+--------------------+-------+---------+------+--------------------+-------------+
| key|               value|  topic|partition|offset|           timestamp|timestampType|
+----+--------------------+-------+---------+------+--------------------+-------------+
|null|[7B 22 69 64 22 3...|persons|        0|     0|2025-10-07 11:57:...|            0|
|null|[7B 22 69 64 22 3...|persons|        0|     1|2025-10-07 11:57:...|            0|
|null|[7B 22 69 64 22 3...|persons|        0|     2|2025-10-07 11:57:...|            0|
|null|[7B 22 69 64 22 3...|persons|        0|     3|2025-10-07 11:57:...|            0|
|null|[7B 22 69 64 22 3...|persons|        0|     4|2025-10-07 11:57:...|            0|
|null|[7B 22 69 64 22 3...|persons|        0|     5|2025-10-07 11:57:...|            0|
|null|[7B 22 69 64 22 3...|persons|        0|     6|2025-10-07 11:57:...|            0|
|null|[7B 22 69 64 22 3

False

# Afficher uniquement la valeur (value) du message Kafka en string

In [45]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("SparkStreamingDemo").getOrCreate()
df_stream = spark.readStream.format("kafka") \
    .option('kafka.bootstrap.servers', kafka_server) \
    .option('subscribe', topic) \
    .option('startingOffsets', 'earliest') \
    .load()

# Afficher uniquement la valeur (value) du message Kafka en string
lines = df_stream.selectExpr("CAST(value AS STRING)")

query = lines.writeStream.format("console").start()
query.awaitTermination(10)

25/10/07 12:05:07 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-a877310a-f9aa-412b-aa9f-46d9c34b2dd5. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/10/07 12:05:07 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+--------------------+
|               value|
+--------------------+
|{"id": 1, "name":...|
|{"id": 2, "name":...|
|{"id": 3, "name":...|
|{"id": 4, "name":...|
|{"id": 5, "name":...|
|{"id": 6, "name":...|
|{"id": 7, "name":...|
|{"id": 8, "name":...|
|{"id": 9, "name":...|
|{"id": 10, "name"...|
|{"id": 11, "name"...|
|{"id": 12, "name"...|
|{"id": 13, "name"...|
|{"id": 14, "name"...|
|{"id": 15, "name"...|
|{"id": 16, "name"...|
|{"id": 17, "name"...|
|{"id": 18, "name"...|
|{"id": 19, "name"...|
|{"id": 20, "name"...|
+--------------------+
only showing top 20 rows



False

# Faire une execution ligne par ligne

In [46]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("SparkExample3").getOrCreate()
df_stream = spark.readStream.format("kafka") \
    .option('kafka.bootstrap.servers', kafka_server) \
    .option('subscribe', topic) \
    .option('startingOffsets', 'earliest') \
    .load()

def process_batch(batch_df, batch_id):
    batch_df.write.format("jdbc") \
        .option("url", "jdbc:postgresql://postgres:5432/events") \
        .option("dbtable", "public.raw_persons") \
        .option("user", "app") \
        .option("password", "1234") \
        .mode("append") \
        .save()

query = df_stream.writeStream \
    .foreachBatch(process_batch) \
    .start()
query.awaitTermination(10)
query.stop()

25/10/07 12:05:24 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
25/10/07 12:05:24 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-850de7a9-97d2-4e7e-bf44-86ef2248e7cf. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/10/07 12:05:24 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


# Formater les données JSON reçues

In [47]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

spark = SparkSession.builder.appName("SparkExample3").getOrCreate()
df_stream = spark.readStream.format("kafka") \
    .option('kafka.bootstrap.servers', kafka_server) \
    .option('subscribe', topic) \
    .option('startingOffsets', 'earliest') \
    .load()

schema = StructType([
    StructField("id", IntegerType(), False),
    StructField("name", StringType(), False),
    StructField("age", IntegerType(), False),
    StructField("city", StringType(), False)
])

lines = df_stream.selectExpr("CAST(value AS STRING) as json_str") \
    .select(from_json(col("json_str"), schema).alias("data")) \
    .select("data.*")

def process_batch(batch_df, batch_id):
    batch_df.write.format("jdbc") \
        .option("url", "jdbc:postgresql://postgres:5432/events") \
        .option("dbtable", "public.persons2") \
        .option("user", "app") \
        .option("password", "1234") \
        .mode("append") \
        .save()

query = lines.writeStream \
    .foreachBatch(process_batch) \
    .start()
query.awaitTermination(10)
query.stop()

25/10/07 12:06:22 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-ad3272fa-a0db-44d6-86bf-9be9cd8761bb. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/10/07 12:06:22 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [54]:
silver_to_kafka = lines.select(col('id').cast('string').alias('value')) \
    .writeStream.format('kafka') \
    .option('kafka.bootstrap.servers', 'kafka:9092') \
    .option('topic', 'persons-silver') \
    .option('checkpointLocation', '/tmp/checkpoint') \
    .start()
    
silver_to_kafka.awaitTermination(10)


25/10/07 13:18:09 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/10/07 13:18:10 WARN NetworkClient: [Producer clientId=producer-1] Error while fetching metadata with correlation id 1 : {persons-silver=LEADER_NOT_AVAILABLE}


False

In [63]:
df_persons = spark.read.format('jdbc')\
        .option("url", "jdbc:postgresql://postgres:5432/events") \
        .option("dbtable", "public.persons2") \
        .option("user", "app") \
        .option("password", "1234") \
        .load()

df.show(10)

+---+---------+---+----------+
| id|     name|age|      city|
+---+---------+---+----------+
|  1|    Alice| 30|     Paris|
|  2|      Bob| 25|      Lyon|
|  3|   Céline| 35| Marseille|
|  4|    David| 28|     Paris|
|  5|     Emma| 40|  Bordeaux|
|  6| François| 22|      Nice|
|  7|Gabrielle| 31|Strasbourg|
|  8|     Hugo| 27|     Lille|
|  9|     Inès| 29|    Nantes|
| 10|   Julien| 33|  Toulouse|
+---+---------+---+----------+
only showing top 10 rows



In [62]:
from pyspark.sql import functions as F

pers = df_persons.filter(F.col('name').rlike('^[Aa]'))

pers.show()

+---+------+---+----------+
| id|  name|age|      city|
+---+------+---+----------+
|  1| Alice| 30|     Paris|
| 27|Adrien| 33|Strasbourg|
|  1| Alice| 30|     Paris|
| 27|Adrien| 33|Strasbourg|
+---+------+---+----------+

